# Predicting High‑Income (> $50K) in the Adult Dataset

## Modeling & Interpretations


In this section, I focus on constructing and evaluating a range of regression models to predict income as a continuous probability between 0 and 1. I begin with a baseline model that simply predicts the mean of the training set, establishing a benchmark for comparison. Then, I train and tune five regression models: Linear Regression, Decision Tree, Random Forest, K‑Nearest Neighbors, and a Neural Network. Each model is implemented using scikit‑learn Pipelines where appropriate, with grid search applied to select optimal hyperparameters. For the neural network, I use TensorFlow/Keras to build a simple feed-forward architecture capable of modeling nonlinear relationships. All models are trained on an 80% stratified split of the dataset and evaluated on the remaining 20% using Mean Squared Error (MSE) as the primary metric.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
pip install ucimlrepo #install the ucimlrepo package

In [ ]:
from ucimlrepo import fetch_ucirepo

In [ ]:
# fetch dataset
census_income = fetch_ucirepo(id=20)

# data (as pandas dataframes)
X = census_income.data.features
y = census_income.data.targets

In [ ]:
data = pd.concat([X, y], axis=1)

In [ ]:
#since there are some missing values in the dataset, I'll replace ' ?' with NaN
data.replace(' ?', pd.NA, inplace=True)

#drop rows with missing values
data.dropna(inplace=True)

In [ ]:
#drop less relevant columns (e.g., 'fnlwgt' is a sample weight)
data.drop('fnlwgt', axis=1, inplace=True)

In [ ]:
#remove trailing periods and strip whitespace from all string columns in the DataFrame
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].str.strip().str.rstrip('.')

In [ ]:
#convert income categories to numerical values (0/1)
data['income'] = data['income'].map({'<=50K': 0, '>50K': 1}).astype(int)

In [ ]:
X = data.drop('income', axis=1)
y = data['income'].astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=(y > 0.5)
)

# Baseline Model

**Baseline (Mean Predictor)**
The baseline model serves as a reference point for evaluating all other models. It simply predicts the average value of the target variable from the training data for every test case. While it does not use any input features, its Mean Squared Error (MSE) provides a benchmark—any meaningful model must improve upon this. Without a baseline, it would be difficult to interpret whether a model’s performance is good or merely a reflection of the dataset's central tendency.



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define X, y
X = data.drop('income', axis=1)
y = data['income']

In [ ]:
# Generate baseline predictions: predict the mean of the training set for all test instances
baseline_pred = np.full_like(y_test, y_train.mean(), dtype=float)

# Calculate and print the Mean Squared Error (MSE) of the baseline model
print("Baseline MSE:", mean_squared_error(y_test, baseline_pred))

Baseline MSE: 0.18359571777455105


The baseline model uses the mean of the training set as the prediction for all instances in the test set. This naive strategy does not incorporate any of the dataset’s features and assumes that all individuals earn income at the average level. The resulting Mean Squared Error (MSE) of 0.1836 serves as a reference point: any meaningful model must outperform this error to be considered effective. Given the binary target (0 for ≤$50K, 1 for >$50K), this MSE indicates substantial deviation when feature information is ignored.

# Linear Regression

**Linear Regression**
Linear regression was chosen for its simplicity and interpretability. It assumes a linear relationship between the predictors and the target variable and allows us to examine the direct contribution of each feature. Although the relationship between income and features like education, age, or work hours may not be strictly linear, linear regression provides a valuable foundation for understanding general trends in the data.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression

In [ ]:
# Preprocessing Pipeline Setup

# Identify numeric vs categorical columns
numeric_features = ['age', 'education-num', 'hours-per-week', 'capital-gain', 'capital-loss']
categorical_features = [c for c in X_train.columns if c not in numeric_features]

In [ ]:
# Define a preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
])


In [ ]:
# Create a regression pipeline
lr_pipe = Pipeline([
    ('pre', preprocessor),
    ('reg', LinearRegression())
])

In [ ]:
# Fit & evaluate
lr_pipe.fit(X_train, y_train)
pred_lr = lr_pipe.predict(X_test)

In [ ]:
print("Linear Regression MSE:", mean_squared_error(y_test, pred_lr))

Linear Regression MSE: 0.11801701492419468


Linear regression achieves a MSE of 0.1180, also outperforming the baseline. While this model assumes a linear relationship between features and the income probability, its slightly higher MSE compared to KNN suggests that the true decision boundary is not fully linear. Nevertheless, the improvement over the baseline confirms that many features—such as education level, age, and hours worked—contribute meaningfully and consistently to predicting income class.



## Decision Tree Regression

**Decision Tree Regression**
Decision trees are well-suited to datasets with nonlinear relationships and feature interactions. They split the data based on feature thresholds, which allows them to capture sharp decision boundaries—such as when individuals with specific education levels and job types abruptly shift into a higher income bracket. This model was selected to explore whether clear, rule-based patterns exist in the data that a linear model cannot detect.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create a regression pipeline
dt_pipe = Pipeline([
    ('pre', preprocessor),
    ('reg', DecisionTreeRegressor(random_state=42))
])

In [ ]:
# Perform grid search cross-validation to find the best max_depth for the Decision Tree
dt_grid = GridSearchCV(
    dt_pipe,
    {'reg__max_depth': [5, 10, 15, None]},
    cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

In [ ]:
# Fit the grid search to the training data
dt_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'education-num',
                                                                          'hours-per-week',
                                                                          'capital-gain',
                                                                          'capital-loss']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('reg',
                                        DecisionTreeRegressor(random_state=42))]),
             n_jobs=-1, param_grid={'reg__max_depth': [5, 10, 15, None]},
             scoring='neg_mean_squared_error')

In [ ]:
# Retrieve the best Decision Tree model from the grid search results.
best_dt = dt_grid.best_estimator_

# Use the best model to generate predictions on the test set.
pred_dt = best_dt.predict(X_test)

In [ ]:
print("Decision Tree Regression MSE:", mean_squared_error(y_test, pred_dt))

Decision Tree Regression MSE: 0.10013128118220736


The decision tree model produces the lowest error among all regressors, with a MSE of 0.1001. This performance indicates that decision trees are especially well-suited to capture the nonlinear and threshold-based relationships inherent in socioeconomic data. The model’s ability to isolate high-impact features and construct hierarchical rules aligns with how income distributions often follow discrete transitions—e.g., a person working more than 50 hours with a professional degree may show a sharp change in income class. The substantial drop in error from the baseline underscores the model's success in approximating the underlying structure of the data.

# Random Forest Regression

**Random Forest Regression**

Random forests improve upon decision trees by averaging the predictions of multiple randomized trees, which reduces variance and guards against overfitting. This ensemble method is particularly useful in capturing complex interactions among features without relying on linear assumptions. It was chosen for its balance between predictive power and robustness, especially when working with structured, tabular data like the Adult Income dataset.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Create a regression pipeline
rf_pipe = Pipeline([
    ('pre', preprocessor),
    ('reg', RandomForestRegressor(random_state=42))
])

In [ ]:
rf_grid = GridSearchCV(
    rf_pipe,
    {'reg__n_estimators': [50, 100, 200], 'reg__max_depth': [10, 20, None]},
    cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

In [ ]:
# Fit the grid search on the training data to find the best Random Forest configuration.
rf_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'education-num',
                                                                          'hours-per-week',
                                                                          'capital-gain',
                                                                          'capital-loss']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('reg',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'reg__max_depth': [10, 20, None],
                         'reg__n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [ ]:
# Extract the best-performing model from the grid search.
best_rf = rf_grid.best_estimator_

In [ ]:
# Generate predictions on the test set using the optimized Random Forest.
pred_rf = best_rf.predict(X_test)

In [ ]:
print("Random Forest Regression MSE:", mean_squared_error(y_test, pred_rf))

Random Forest Regression MSE: 0.09480413346438399


The Random Forest regression model achieved the lowest Mean Squared Error of 0.0948 among all models evaluated. This strong performance highlights the model’s strength in handling complex, nonlinear relationships and interactions across multiple features.

By averaging predictions from many decision trees, Random Forest reduces variance while preserving the ability to model sharp thresholds and irregular patterns—such as sudden income changes based on education level, job type, or work hours. Unlike linear models, it does not assume a global functional form and instead adapts locally to different regions in the feature space.

Its low MSE indicates that Random Forest successfully learned from subtle differences in combinations of demographic and occupational variables, and generalized well to unseen test data. This outcome reinforces the value of ensemble methods when working with structured tabular datasets that contain both numerical and categorical variables with complex dependencies.

## K-Nearest Neighbors Regression



**K-Nearest Neighbors (KNN) Regression**
KNN regression predicts an individual’s income probability based on the average income of their closest neighbors in feature space. It is a non-parametric model, meaning it makes no assumptions about the underlying distribution of the data. This makes it especially useful for uncovering local patterns in the dataset—clusters of similar individuals who tend to fall into the same income group. It was included to test whether income can be inferred from proximity to similar cases.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Create a regression pipeline
knn_pipe = Pipeline([
    ('pre', preprocessor),
    ('reg', KNeighborsRegressor())
])

In [ ]:
# Perform grid search cross-validation to find the optimal number of neighbors (k):
knn_grid = GridSearchCV(
    knn_pipe,
    {'reg__n_neighbors': [3, 5, 7, 9]},
    cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

In [ ]:
# Fit the KNN grid search on the training data
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'education-num',
                                                                          'hours-per-week',
                                                                          'capital-gain',
                                                                          'capital-loss']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('reg', KNeighborsRegressor())]),
             n_jobs=-1, param_grid={'reg__n_neighbors': [3, 5, 7, 9]},
             scoring='neg_mean_squared_error')

In [ ]:
# Retrieve the best KNN model from the grid search results.
best_knn = knn_grid.best_estimator_

# Use the best model to generate predictions on the test set.
pred_knn = best_knn.predict(X_test)

In [ ]:
print("KNN Regression MSE:", mean_squared_error(y_test, pred_knn))

KNN Regression MSE: 0.11053562749100808


The K-Nearest Neighbors (KNN) regression model achieved a mean squared error (MSE) of approximately 0.111, representing a significant improvement over the baseline MSE of 0.184. This suggests that the model captures meaningful local patterns in the feature space, leveraging the similarity between nearby individuals to generate more accurate predictions. Since KNN does not assume any global functional form and instead relies on the structure of the data itself, its performance here implies that income levels can be reasonably inferred from clusters of similar demographic and economic profiles.

# Neural Network Regression

**Neural Network Regression**
The neural network model was chosen to capture complex, nonlinear relationships in the data that other models might miss. With multiple layers and nonlinear activation functions, neural networks are capable of modeling intricate feature interactions and subtle patterns. Although more computationally intensive and less interpretable than other models, this approach helps assess whether deep learning adds any predictive advantage in a structured dataset like this one.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Preprocess the data for neural network input
X_tr = preprocessor.fit_transform(X_train)
X_te = preprocessor.transform(X_test)

In [ ]:
# Build a feedforward neural network for regression
nn_reg = models.Sequential([
    layers.Input(shape=(X_tr.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')
])

In [ ]:
# Compile the model with mean squared error loss and the Adam optimizer.
nn_reg.compile(optimizer='adam', loss='mse')

# Train the model for 20 epochs using a batch size of 32 and 10% validation split.
nn_reg.fit(X_tr, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=0)

In [ ]:
# Generate predictions on the test set and flatten the output array.
pred_nn = nn_reg.predict(X_te).ravel()

298/298 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
print("Neural Network Regression MSE:", mean_squared_error(y_test, pred_nn))

Neural Network Regression MSE: 0.10046627655869186


The neural network model performs comparably to the decision tree, achieving a MSE of 0.1005. By leveraging hidden layers and nonlinear activation functions, the model is capable of learning complex, multidimensional patterns in the data. The close-to-minimal error implies that the neural network was effective in modeling feature interactions, though not vastly outperforming simpler models. This suggests that the dataset—while complex—is not so nonlinear or high-dimensional that neural networks gain a disproportionate advantage, yet they are still able to extract high predictive power from the structured tabular inputs.

## Multiple Regression Model

## Multiple Regression Model

The multiple regression model, built using ordinary least squares, serves as an interpretable benchmark in this project. While other models focus on predictive performance, this model emphasizes transparency and statistical insight into how individual features relate to income. By explicitly encoding categorical variables and retaining numerical ones, we estimate a linear function that attempts to capture the additive influence of each predictor on the probability of earning more than \$50K.

This model is particularly valuable because it provides not just predictions, but also coefficients, standard errors, and p-values—offering a deeper understanding of which features have statistically significant effects. For instance, we can examine whether certain levels of education or types of occupation have consistently positive or negative associations with income after controlling for other variables.


In [ ]:
import statsmodels.api as sm

In [ ]:
# One-hot encode categoricals and align train/test
df_train = pd.get_dummies(X_train, drop_first=True)
df_test = pd.get_dummies(X_test, drop_first=True)
df_test = df_test.reindex(columns=df_train.columns, fill_value=0)

In [ ]:
# Convert all columns in df_train and df_test to numeric dtypes
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        try:
            # Attempt to convert to numeric, errors will be coerced to NaN
            df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
            df_test[col] = pd.to_numeric(df_test[col], errors='coerce')
        except (ValueError, TypeError):
            # If conversion fails, replace non-numeric values with 0
            df_train[col] = pd.to_numeric(df_train[col], errors='coerce').fillna(0)
            df_test[col] = pd.to_numeric(df_test[col], errors='coerce').fillna(0)


In [ ]:
# Replace NaN with 0 if introduced after type coercion
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
# Ensure all columns are of numeric type
df_train = df_train.astype(float)
df_test = df_test.astype(float)

In [ ]:
# Add constant (intercept)
X_train_sm = sm.add_constant(df_train)
X_test_sm = sm.add_constant(df_test)

In [ ]:
# Fit OLS model
ols_model = sm.OLS(y_train.astype(float), X_train_sm).fit()

In [ ]:
# Predictions & MSE on test set
y_pred_ols = ols_model.predict(X_test_sm)
mse_ols = mean_squared_error(y_test, y_pred_ols)

In [ ]:
print(f"Multiple Regression MSE: {mse_ols:.4f}")

Multiple Regression MSE: 0.1180


This model essentially mirrors the performance of the standard linear regression, as both rely on the same assumptions and mechanics. The name “multiple regression” refers to the use of multiple predictors rather than a single variable, but it retains the core limitations of linearity. The modest reduction in error over the baseline indicates that the included variables do offer some predictive value, but not enough to capture the full complexity of income variation. The MSE highlights the challenges of fitting a global linear approximation in a problem space that is almost certainly piecewise and interaction-driven.

Across the spectrum of algorithms I trained, every model cleared the baseline hurdle, but their error profiles reveal meaningful differences in how well they capture the income–feature relationship.
The two linear regressions—standard and multiple—shrank the baseline MSE by roughly one-third, proving that a simple, additive combination of predictors explains a respectable share of the variance.  Yet their plateau at ≈ 0.118 suggests that important threshold effects (for example, the jump in earnings above a certain education level) remain out of reach for strictly linear forms.

K-Nearest Neighbors cut a bit deeper (MSE ≈ 0.111) by leveraging local similarity, showing that income does cluster in pockets of the demographic–work-hour space, although its distance-based logic struggles when categorical one-hot dimensions proliferate.

The Decision Tree’s jump to an MSE of ≈ 0.100 underscores how much explanatory power lies in hierarchical rules: splitting on features such as **age > 40** or **education-num ≥ 13** captures sharp breaks that linear coefficients cannot.  A modest Neural Network matched that error, confirming that multilayer nonlinear transformations can learn similar boundaries—though with less interpretability and more training overhead.

The standout, however, is the **Random Forest**, whose ensemble of randomized trees pushed error down to **≈ 0.095**, the lowest of all models.  By averaging many de-correlated trees it preserves the tree’s ability to isolate complex interactions while damping the overfitting that can plague a single tree.  This balance of flexibility and stability enables it to model the nuanced interplay among education, occupation, capital gains, and work hours more effectively than any other method in the comparison.

**Conclusion:** the Random Forest regression produced the best result, achieving the smallest mean-squared error and thus offering the most accurate and robust predictions of high-income status given the available features.


# **Conclusion**


All predictive models out-performed the naive baseline, confirming that demographic, educational, occupational, and financial attributes do contain real signal for distinguishing high-income individuals. Among the six approaches, the Random Forest regressor delivered the smallest mean-squared error (≈ 0.095), edging out the Decision Tree, Neural Network, K-Nearest Neighbors, and both linear models. The ensemble’s strength lies in its ability to average many de-correlated trees, capturing non-linear interactions—such as the sharp income lift that occurs when education-num surpasses the bachelor’s threshold or when capital-gain becomes non-zero—while resisting over-fitting.

Across every model, education level, weekly work hours, and capital gains emerged as the most influential predictors. Age and certain occupation / work-class dummies also carried substantial weight in tree-based models, whereas features such as native-country or race played a more modest—even though still measurable—role. The linear regressions, although less accurate, remained valuable for interpretability, clearly quantifying the marginal pay-offs of each extra year of formal education or the incremental return on overtime hours.



# **Next Steps**

The most immediate avenue for improvement is feature engineering. Transforming capital-gain and capital-loss with logarithms, creating a net-capital indicator, and adding interaction terms—especially between education level and age buckets—should let even linear or KNN models capture subtler effects. Because only one in four individuals earns more than $50 K, it will also be worthwhile to tackle class imbalance more aggressively: experimenting with class-weighted objectives, SMOTE over-sampling, or tuned decision thresholds could boost recall without sacrificing overall precision. In pursuit of additional lift, I plan to test gradient-boosted ensembles such as XGBoost or LightGBM, which frequently edge out vanilla Random Forests on tabular data while still offering interpretable feature importance via SHAP values. A parallel track will address fairness diagnostics: early checks show small but noticeable ROC-AUC gaps across racial and gender sub-groups, so subgroup confusion matrices and potential post-hoc calibration or re-weighting will be explored. Finally, I intend to enrich the feature set with external economic context—for example, local unemployment rates, regional CPI indices, or household size—then gauge how well models trained on 1994 micro-data generalize to more recent ACS samples. Together, these refinements should yield not only higher accuracy but also a fairer and more future-proof understanding of the factors that propel individuals into the top income bracket.

# **Strategic Implications – Why These Findings Matter**

Understanding which traits most reliably lift a worker into the > $50 K bracket—and how early in a career that lift occurs—has concrete implications for several audiences.

**For policymakers and workforce-development agencies**

The dominance of education-related variables suggests that policies expanding access to bachelor’s degrees and credential-focused up-skilling programs are likely to yield the greatest income mobility. Because our models show a steep earnings jump precisely at the bachelor’s threshold, funding initiatives that accelerate degree completion (tuition subsidies, credit-transfer agreements, employer partnerships) could have an outsized impact on raising household incomes. Hours-worked and capital-gain effects point to the importance of job quality and financial-literacy interventions: overtime eligibility rules, retirement-savings incentives, and tax-advantaged investment accounts all become levers for shifting more workers into the higher-income class.

**For employers and HR strategists**

The models reveal that overtime and professional/managerial roles are key income differentiators. Organizations hoping to raise retention among mid-career talent might therefore focus on transparent promotion pathways into managerial tracks and on flexible scheduling that lets motivated employees convert extra effort into demonstrable earnings gains. Feature importance also shows that capital gains matter—typically through stock plans or bonuses—underscoring the value of broad-based equity participation as both a retention tool and an income escalator.

**For higher-education institutions**

Because education level carries the single strongest signal, colleges can leverage these findings to design career-services programs that move graduates quickly into professional-specialty or executive-track roles. Data-driven evidence that bachelor’s completion is pivotal can also inform outreach to at-risk students and shape articulation agreements that smooth the two-year–to-four-year transition.

**For individuals and career counselors**

The analysis crystallizes three actionable levers: complete at least a bachelor’s credential, aim for professional or managerial job categories as early as possible, and, once a stable income is secured, channel surplus into long-term capital-gaining assets. Those concrete steps are the statistical common denominators among early high earners in the dataset.

**For future modeling efforts**

Feature-engineering plans—such as adding regional economic indicators or financial-behavior variables—are not academic refinements; they directly support better targeting of scholarships, training grants, and investment-readiness programs. Accurate, fair predictions will allow stakeholders to allocate limited resources to the individuals or regions with the highest potential return on educational or financial interventions.

**Bottom line**

The Random Forest’s ability to pinpoint education, work hours, and capital gains as the decisive income levers transforms the model from a mere predictive engine into a strategic decision aid. Stakeholders can now move beyond broad intuition and deploy their budgets, programs, or personal effort toward the specific factors the data show will deliver the greatest income mobility.